# Content-based Recommendation

Content-based recommender algorithms use the content of the items for making a recommendation. For example, for movie recommendation movie contents such as plot summary, director, casting, jenres, release date, etc. and user content such as previously watched movies, gender, age, etc. can be used to find out which movies can be recommended to the users.
(by Tevfik Aytekin)

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix
from collections import Counter
from sklearn.metrics import pairwise_distances
import copy
import heapq
import sys, os
import pickle


### Movielens ml-latest-small dataset

In [5]:
with open('../../datasets/ml-latest-small/README.txt', 'r') as f:
    print(f.read())

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

In [6]:
ratings = pd.read_csv("../../datasets/ml-latest-small/ratings.csv", sep=",")
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
links = pd.read_csv("../../datasets/ml-latest-small/links.csv", sep=",")
print(links.shape)
links.head()

(9742, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
movies = pd.read_csv("../../datasets/ml-latest-small/movies.csv", sep=",")
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
tags = pd.read_csv("../../datasets/ml-latest-small/tags.csv", sep=",")
print(tags.shape)
tags.head()

(3683, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## Create User Item Rating Map
It might take some time but will be useful later.

In [11]:
rating_map = {}
for i in range(len(ratings)):
    key = str(ratings.iloc[i,0]) + '_' +str(ratings.iloc[i,1])
    rating_map[key]=ratings.iloc[i,2]

In [12]:
iterator = iter(rating_map.items())
for i in range(5):
    print(next(iterator))

('1_1', 4.0)
('1_3', 4.0)
('1_6', 4.0)
('1_47', 5.0)
('1_50', 5.0)


## Create movie genre map
This map will also be useful later

In [13]:
movie_genres = {}
for i in range(len(movies)):
    key = movies.iloc[i,0]
    movie_genres[key]=movies.iloc[i,2].split('|')

In [14]:
iterator = iter(movie_genres.items())
for i in range(5):
    print(next(iterator))

(1, ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'])
(2, ['Adventure', 'Children', 'Fantasy'])
(3, ['Comedy', 'Romance'])
(4, ['Comedy', 'Drama', 'Romance'])
(5, ['Comedy'])


## Rating Prediction

### Algorithm 
 
Predict rating of user $u$ for item $i$
- Calculate the similarity of items that are rated by $u$ with $i$.
- Use these similarities to calculate a weighted average of the ratings.

Below we only use the genres to calculate the content similarity between movies, but in general, one can use many other information such as the director, date, casting and plot summary. In order to do this one needs to find a way to represent this information and a similarity metric to quantify the similarity.

Let us first see an example. 

In [405]:
movies_ratings_join = movies.join(ratings, on="movieId", lsuffix='_caller', rsuffix='_other')

In [408]:
movies_ratings_join.query("userId == 1").head(5)


,movieId_caller,title,genres,userId,movieId_other,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,3.0,4.0,964981247.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1.0,6.0,4.0,964982224.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1.0,47.0,5.0,964983815.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1.0,50.0,5.0,964982931.0
4,5,Father of the Bride Part II (1995),Comedy,1.0,70.0,3.0,964982400.0


What would be the rating of this user for the following movie?

In [417]:
movies_ratings_join.query("userId == 1 and movieId_caller == 60")


,movieId_caller,title,genres,userId,movieId_other,rating,timestamp
53,60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,1.0,1073.0,5.0,964981680.0


Following is the calculation for this example:

Jaccard(Toy Story, Indian in the Cupboard) = ? <br>
Jaccard(Jumanji, Indian in the Cupboard) = ? <br>
Jaccard(Grumpier Old Men, Indian in the Cupboard) = ? <br>
Jaccard(Waiting to Exhale, Indian in the Cupboard) = ? <br>
Jaccard(Father of the Bride Part II, Indian in the Cupboard) = ?<br>

Weighted average = 



### Jaccard Similarity

Given two sets $A$ and $B$,

$Jaccard(A, B) = \frac{|A \cap B|}{|A \cup B|}$

For example if $A = \{a, b, c, d\}$ and $B = \{b, d, e ,f, g\}$ then

$Jaccard(A, B) = \frac{2}{7}$

In [15]:
def content_based_rating_prediction(u, i):
    r = 0
    sum_sim = 0
    # find the movies rated by u
    movies = ratings[ratings["userId"]==u].movieId
    for j in movies:
        sim = genre_sim(i, j)
        key = str(u)+"_"+str(j)
        r += sim*rating_map[key]
        sum_sim += sim
    if sum_sim == 0:
        return 0
    else:
        return r / sum_sim        

In [16]:
# finds the genre similary of items i and j using Jaccard similarity
def genre_sim(i,j):
    genres_i = movie_genres[i]
    genres_j = movie_genres[j]
    #print(genres_i)
    #print(genres_j)
    intersection_size = len(set(genres_i).intersection(genres_j))
    union_size = len(set(genres_i).union(genres_j))
    return intersection_size / union_size
    

In [552]:
# finds the genre similary of items i and j using Jaccard similarity
def genre_sim2(i,j):
    x = item_content_matrix.loc[i].to_numpy()
    y = item_content_matrix.loc[j].to_numpy()
    intersection_size = np.count_nonzero(np.bitwise_and(x,y))
    union_size = np.count_nonzero(np.bitwise_or(x,y))
    return intersection_size / union_size


## Evaluation

How can we measure the performance of a recommender algorithm? This is similar to the evaluation used in machine learning.

In [258]:
X_train, X_test = train_test_split(ratings, test_size=1000)
train_size = X_train.shape[0]
test_size = X_test.shape[0]
print("Test size:", test_size)
error = 0
for k in range(test_size): 
    u = X_test.iloc[k,0]
    i = X_test.iloc[k,1]
    r = X_test.iloc[k,2]
    error += np.abs(r - content_based_rating_prediction(u,i))
print(error/test_size)

Test size: 1000
0.6798966866853725


## Top-N recommendation
The task in top-$N$ prediction is to recommend $N$ items to a user. 


Recommend $N$ movies to user $u$
- Predict the ratings of all items which are not watched by $u$
- Sort the predicted ratings
- Recommend the movies with the highest predicted ratings

In [36]:
x  = ratings.query("userId != 1").movieId.unique()
len(x)

9723

In [39]:
def top_N_rec1(u):
    preds = pd.Series([], dtype='float')
    # find the movies not rated by u
    movies_not_rated = ratings.query("userId != @u").movieId.unique()
    for m in movies_not_rated:
        preds[m] = content_based_rating_prediction(u, m)
    return preds.sort_values(ascending=False)[:10]    

In [40]:
top_N_rec1(1)

7335      5.000000
4426      5.000000
2066      5.000000
72603     4.718136
170597    4.718136
163386    4.718136
163112    4.718136
153236    4.718136
193573    4.718136
151769    4.718136
dtype: float64

## Efficiency Issues

There are important inefficiencies in this algorithm:

- The algorithm predicts the rating of all items which are not rated by the user. In the case of millions of items this algorithm is practically infeasible. Numerous techniques have been developed to remedy this problem. Can you suggest a solution? 
- In rating prediction, similarity between target item and items rated by the user are calculated. To make a recommendation to another user similarity calculations will be done again. For making recommendations to users in general many similarity calculations will be repeated. A general solution to this problem is to precalculate the similarities between items. Moreover, you don't need to store all similarities, only storing $k$ most similar items to every item will be enough. Size of $k$ can be determined according to the needs.


## Building an item similarity table
This table will hold the most similar $k$ items for each item. In order to build this table we need to calculate all pairwise similarities which takes $O(n^2)$ time. There is no escape from this $O(n^2)$ time unless you use an approximation algorithm such as LSH (Locality Sensitive Hashing) for nearest neighbor search. 

## Efficiency Improvements

### Create movie similarity matrix first

Given $n$ items finding all pairwise similarities requires $O(n^2)$ similarity calculations which can take a lot of time if $n$ is large as the following code illustrates.

In [571]:
pq =[1,20,3]
type(pq)
heapq.heappush(pq, 8)
heapq.heapify(pq)
heapq.nsmallest(1,pq)

[1]

In [575]:
K = 10
similarity_map = {}
movie_ids = movies['movieId'].to_numpy()
print(len(movie_ids))
for i in range(len(movie_ids)):
    if (i % 100 == 0):
        print(i)
    if i not in similarity_map:
        similarity_map[i] = []
    pq = similarity_map[i]
    for j in range(i+1, len(movie_ids)):
        sim = genre_sim(movie_ids[i],movie_ids[j])
        if (len(pq) >= K):
            smallest = pq[0]
            if (sim > smallest[0]):
                heapq.heappop(pq)
                heapq.heappush(pq, (sim, j))
        else:
            heapq.heappush(pq, (sim, j))


9742
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700


In [585]:
similarity_map[1505]

[(1.0, 1577),
 (1.0, 3230),
 (1.0, 2539),
 (1.0, 4014),
 (1.0, 3336),
 (1.0, 5686),
 (1.0, 5087),
 (1.0, 5624),
 (1.0, 4424),
 (1.0, 6944)]